Una red neuronal es un modelo matemático inspirado en el cerebro humano. Está compuesta por nodos llamados neuronas, que están organizados en capas. Estas redes se entrenan para aprender patrones en los datos a través de un proceso llamado propagación hacia adelante y retropropagación.